# Lab 07:  Decision Trees



#### Part 1

Train and fine tune a decision tree for the moons dataset  
* Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset
* Use train_test_split() to split the dataset into a training set and a test set.
* Use grid search with cross-validation (with the help of the GridSearchCV
class) to find good hyperparameter values for a DecisionTreeClassifier.
Hint: try various values for max_leaf_nodes.
* Train it on the full training set using these hyperparameters, and measure
your model’s performance on the test set. You should get roughly 85% to 87%
accuracy.

In [10]:
# Jimmy Davis 2/24/2025
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import mode

# Generate the moons dataset
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use GridSearchCV to find the best hyperparameters
param_grid = {
    'max_leaf_nodes': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}
tree_clf = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(tree_clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Train the model with the best hyperparameters
best_tree_clf = grid_search.best_estimator_
y_pred = best_tree_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Decision tree accuracy: {accuracy:.2%}")

Decision tree accuracy: 87.00%


#### Part 2: Grow a forest by following these steps:

* Continuing the previous exercise, generate 1,000 subsets of the training set,
each containing 100 instances selected randomly. Hint: you can use Scikit-
Learn’s ShuffleSplit class for this.
* Train one Decision Tree on each subset, using the best hyperparameter values
found in the previous exercise. Evaluate these 1,000 Decision Trees on the test
set. Since they were trained on smaller sets, these Decision Trees will likely
perform worse than the first Decision Tree, achieving only about 80%
accuracy.
* Now comes the magic. For each test set instance, generate the predictions of
the 1,000 Decision Trees, and keep only the most frequent prediction (you can
use SciPy’s mode() function for this). This approach gives you majority-vote
predictions over the test set.
* Evaluate these predictions on the test set: you should obtain a slightly higher
accuracy than your first model (about 0.5 to 1.5% higher). Congratulations,
you have trained a Random Forest classifier!

In [11]:
import numpy as np
from collections import Counter

# Generate 1,000 subsets of the training set
n_trees = 1000
n_instances = 100
subsets = ShuffleSplit(n_splits=n_trees, train_size=n_instances, random_state=42)

# Train 1,000 Decision Trees on the subsets
forest = []
for subset_train_idx, _ in subsets.split(X_train):
    X_subset = X_train[subset_train_idx]
    y_subset = y_train[subset_train_idx]
    tree_clf = DecisionTreeClassifier(**best_tree_clf.get_params())
    tree_clf.fit(X_subset, y_subset)
    forest.append(tree_clf)

# Evaluate each tree on the test set
tree_accuracies = []
for tree in forest:
    y_pred = tree.predict(X_test)
    tree_accuracies.append(accuracy_score(y_test, y_pred))
print(f"Average tree accuracy: {np.mean(tree_accuracies):.2%}")

# Majority voting for predictions
y_pred_forest = []
for instance in X_test:
    predictions = [tree.predict([instance])[0] for tree in forest]
    
    # Use Counter to find the majority vote
    majority_vote = Counter(predictions).most_common(1)[0][0]
    y_pred_forest.append(majority_vote)

# Evaluate the random forest
forest_accuracy = accuracy_score(y_test, y_pred_forest)
print(f"Random Forest Accuracy: {forest_accuracy:.2%}")

Average tree accuracy: 80.12%
Random Forest Accuracy: 87.25%
